In [ ]:
lines = sc.textFile('./data/test_gnm/graph.txt',8)

In [ ]:
lines.count()

In [ ]:
lines.getNumPartitions()

In [ ]:
lines.take(5)

In [ ]:
# Converting lines into edges. 
edges = lines.map(lambda l: tuple(map(int,l.split())))
graph = edges.combineByKey(
    lambda e:[e],\
    lambda v,e:v+[e],\
    lambda v1,v2:v1+v2)
graph.cache()

In [ ]:
graph.sortByKey().take(5)

In [ ]:
graph.getNumPartitions()

In [ ]:
G = graph.sortByKey().collect()

In [ ]:
import math
n = graph.count()
m = lines.count()
s = 1
alpha = 0.2
pf = 1./n
delta = 1./n
epsilon = 0.5
w = int(math.ceil(math.log(1./pf)/(epsilon*epsilon*delta)))
k = 10

In [ ]:
G

In [ ]:
import random
import math

def find_vertex(G,v):
    start = 0
    end = len(G)-1
    while (start<=end):
        middle = (start+end)/2
        if (G[middle][0]==v):
            return middle
        elif (v<G[middle][0]):
            end = middle-1
        else:
            start = middle+1
    return -1
def random_walk(v,G):
    current=v
    while (random.random()>alpha):
        idx = find_vertex(G,current)
        if (idx==0):
            break
        Nout=len(G[idx][1])
        if (Nout==0):
            break
        choice = int(math.floor(random.random()*Nout))
        current = G[idx][1][choice]
    yield current

In [ ]:
random_walk(1,G)

In [ ]:
iterations = sc.parallelize([s for i in range(w)],8)

In [ ]:
iterations.getNumPartitions()

In [ ]:
from operator import add
counts = iterations.flatMap(lambda x: random_walk(x,G)).map(lambda x: (x,1)).reduceByKey(add).cache()

In [ ]:
results = counts.sortByKey().map(lambda (x,y):(y,x)).sortByKey(False).map(lambda (x,y):(y,x)).mapValues(lambda v: v*1./w)

In [ ]:
results.take(k)